In [1]:
#VM DATASET INIT SET UP
!pip install kaggle
import json
import os
from pathlib import Path
import pandas as pd
from torchvision.transforms import v2
from torchvision import transforms, datasets

#your api key
api_key = {"username":"ishag7",
           "key":"73dac2ca88ce68fa7aaf9390f36326a5"}

os.environ['KAGGLE_USERNAME'] = "ishag7"
os.environ['KAGGLE_KEY'] = "73dac2ca88ce68fa7aaf9390f36326a5"

#uses pathlib Path
kaggle_path = Path('/home/jupyter/.kaggle')

#make kaggle dir
os.makedirs(kaggle_path)

#opens file and dumps python dict to json object
with open (kaggle_path/'kaggle.json', 'w') as handl:
           json.dump(api_key,handl)
#change dir permissions
os.chmod(kaggle_path/'kaggle.json', 700)
#import kaggle api
import kaggle as kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
#set up api and authentication wit json token
api = KaggleApi()
api.authenticate()
#download train and test files
kaggle.api.dataset_download_files('birdy654/cifake-real-and-ai-generated-synthetic-images', unzip = True)

!pip install wandb
import random as random
random.seed(420)
val_fake = random.sample(os.listdir('/home/jupyter/test/FAKE/'), 5000)
val_real = random.sample(os.listdir('/home/jupyter/test/REAL/'), 5000)
pd.DataFrame(val_fake).to_csv('./val_fake_filenames.csv', header = None, index = None)
pd.DataFrame(val_real).to_csv('./val_real_filenames.csv', header = None, index = None)
os.makedirs('/home/jupyter/val')
os.makedirs('/home/jupyter/val/REAL')
os.makedirs('/home/jupyter/val/FAKE')
for i in range(0, 5000):
    os.rename('/home/jupyter/test/FAKE/' + val_fake[i], '/home/jupyter/val/FAKE/' + val_fake[i])
    os.rename('/home/jupyter/test/REAL/' + val_real[i], '/home/jupyter/val/REAL/' + val_real[i])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 10.7 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.6-py3-none-any.whl size=111943 sha256=175768ebc3e5808904ee636a54173358afbf1fefad159a44b65c10e2849920be
  Stored in directory: /home/jupyter/.cache/pip/wheels/53/34/8c/8ca3450d17206d9e37e1ee3aeb47cbb2873d22a9e0c60eb137
Successfully built kaggle


/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 16.2 MB/s eta 0:00:00


In [25]:
import torch
import torchvision
from torchvision.io import read_image
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.transforms import v2
import torch.nn.functional as F
import wandb as wandb

# Borrowed with modifications from CSE 599G Spring 2023
def check_accuracy_part34(loader, model, device, dtype=torch.float32, num_samples = 10000):
 #   print("Starting check_accuracy_part34")
    num_correct = 0
    model.eval()  # set model to evaluation mode
    criterion = nn.BCEWithLogitsLoss()
    s_fn = nn.Sigmoid()
    with torch.no_grad():
        for t, (x, y) in enumerate(loader):
          #  print("Iteration of internal loop in accuracy checker")
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=dtype)
          #  print("Finished converting x and y to right device. Calculating scores")
            score = model(x)
            loss = criterion(score, y.reshape([-1,1 ]))
            num_correct += (torch.round(s_fn(score)) == y.reshape([-1,1])).float().sum()
          #  print("Finished calculating scores. ")
          #  print("Finished calculating scores & num_correct, num_samples")
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return acc, loss


# Borrowed with modifications from CSE 599G Spring 2023
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

# Borrowed with modifications from CSE 599G Spring 2023
def train_part34(model, optimizer, device, loader, val_loader=None, epochs=1, dtype=torch.float32, print_every=50):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.

    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for

    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    criterion = nn.BCEWithLogitsLoss()
    for e in range(epochs):
        print("epoch ", e)
        # unpacking train_loader is the bottleneck; saving it as an iterable
        # to memory before the loop doesn't help
        for t, (x, y) in enumerate(loader):

           # print("Putting model in training mode")
            model.train()  # put model to training mode
           # print("Converting X and y to correct devices and data types")
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
           # print("X, y conversion complete")

           # print("Generating scores")
            score = model(x)
            #print("Done calculating score. Calculating loss...")
            loss = criterion(score, y.unsqueeze(1).float())            
            #print("Calculated loss. Zeroing out gradients...")
            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

           # print("Zeroed out gradients. Starting backward pass...")
            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()
           # print("Completed backward pass. Making optimizer step...")

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0 and val_loader != None:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                val_acc,val_loss = check_accuracy_part34(val_loader, model, device)
                wandb.log({"acc": val_acc, 
                           "train_loss": loss,
                           "val_loss": val_loss})

def main(data_dir_path = "/home/jupyter/",
         batch_size = 128, num_workers = 2, print_every = 50):
    
    print("Setting torch to run on GPU if available")

    USE_GPU = True
    dtype = torch.float32 

    if USE_GPU and torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    # Constant to control how frequently we print train loss.
    print_every = 50
    print('using device:', device)

    print("Setting up transforms to apply to image data...")
    transforms = v2.Compose([
    v2.ToImagePIL(),  # Convert to tensor, only needed if you had a PIL image
    v2.ToDtype(torch.uint8),  # optional, most input are already uint8 at this point
    v2.ToTensor(),
    #v2.Resize((128,128)),
    #v2.RandomResizedCrop(size=(128, 128), scale = (0.08, 1.0), antialias=True),  # Or Resize(antialias=True)
    #v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
])
    print("Complete.")

    data_dir = data_dir_path

    print("Data directory is set to: ", data_dir)

    print("Setting up ImageFolder objects for train, val, test...")

    train_dataset = datasets.ImageFolder(root=data_dir+'/train/', transform=transforms)
    print("Train complete")
    val_dataset = datasets.ImageFolder(root=data_dir+'/val', transform=transforms)
    print("Val complete")
    test_dataset = datasets.ImageFolder(root=data_dir+'/test/', transform=transforms)
    print("Test complete")

    print("Now setting up loaders linked to those dataset objects:")

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size,
                                            num_workers=num_workers,
                                            shuffle = True)

    print("Train loader complete")

    val_loader = torch.utils.data.DataLoader(val_dataset,
                                            batch_size=batch_size,
                                            num_workers=num_workers,
                                            shuffle = True)

    print("Val loader complete")

    test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=batch_size,
                                            num_workers=num_workers,
                                             shuffle = True)

    print("Test loader complete")



    channel_1 = 32
    channel_2 = 32

    print("Constructing model...")
    model = nn.Sequential(
        nn.Conv2d(3, channel_1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
        nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
        Flatten(),
        # three layers of 64 rectified linear units per Bird, Lotfi (2023)
        nn.Linear(channel_2 * 32 * 32, 64),
        nn.ReLU(),
        nn.Linear(64,64),
        nn.ReLU(),
        nn.Linear(64,1),
    )
    for rate in [1e-3, 1e-4, 1e-5, 1e-6]:
        for l2reg in [0.1, 0.01, 0.001, 0.0001]:
            wandb.init(
            # set the wandb project where this run will be logged
            project="ADAM_runs",
            # track hyperparameters and run metadata
            config={
            "learning_rate": rate,
            "l2reg": l2reg,
            "epochs": 5,
            "optimizier": "ADAM"}
            )
            optimizer = optim.Adam(model.parameters(),
                                   lr = rate,
                                   weight_decay = l2reg)
            train_part34(model, optimizer, device, train_loader, val_loader, epochs=3) 
            wandb.finish()

In [26]:
main()

Setting torch to run on GPU if available
using device: cuda
Setting up transforms to apply to image data...
Complete.
Data directory is set to:  /home/jupyter/
Setting up ImageFolder objects for train, val, test...
Train complete
Val complete
Test complete
Now setting up loaders linked to those dataset objects:
Train loader complete
Val loader complete
Test loader complete
Constructing model...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


epoch  0
Iteration 0, loss = 0.6910
Got 5886 / 10000 correct (58.86)
Iteration 50, loss = 0.6142
Got 6501 / 10000 correct (65.01)
Iteration 100, loss = 0.5701
Got 7164 / 10000 correct (71.64)
Iteration 150, loss = 0.5493
Got 7739 / 10000 correct (77.39)
Iteration 200, loss = 0.5738
Got 7678 / 10000 correct (76.78)
Iteration 250, loss = 0.4625
Got 7834 / 10000 correct (78.34)
Iteration 300, loss = 0.4471
Got 7947 / 10000 correct (79.47)
Iteration 350, loss = 0.4853
Got 7868 / 10000 correct (78.68)
Iteration 400, loss = 0.4959
Got 7634 / 10000 correct (76.34)
Iteration 450, loss = 0.5082
Got 7524 / 10000 correct (75.24)
Iteration 500, loss = 0.4555
Got 7850 / 10000 correct (78.50)
Iteration 550, loss = 0.4885
Got 7895 / 10000 correct (78.95)
Iteration 600, loss = 0.4246
Got 7920 / 10000 correct (79.20)
Iteration 650, loss = 0.5056
Got 7923 / 10000 correct (79.23)
Iteration 700, loss = 0.5268
Got 7865 / 10000 correct (78.65)
Iteration 750, loss = 0.4972
Got 7962 / 10000 correct (79.62)
ep

KeyboardInterrupt: 